In [3]:
import numpy as np

In [17]:
def normalize(input_matrix, axis):
    """
    Normalizes the columns or rows of a 2d input_matrix so they sum to 1
    """

    sums = input_matrix.sum(axis=axis)
    # try:
    #     assert (np.count_nonzero(sums)==np.shape(sums)[0]) # no set should sum to zero
    # except Exception:
    #     raise Exception("Error while normalizing. Sums to zero")
    if axis == 0:
        new_matrix = np.divide(input_matrix, sums[np.newaxis:], out=np.zeros_like(input_matrix),
                               where=sums[np.newaxis:]!=0)
    else:
        new_matrix = np.divide(input_matrix, sums[:, np.newaxis], out=np.zeros_like(input_matrix),
                               where=sums[:, np.newaxis]!=0)
    return new_matrix

# Parameters
## beta
beta as k x |V| matrix that represents probability that word belongs to topic. initialize to 1/k for each word.

## alpha
probability of topic in the whole corpus represented as |k| size matrix

## term doc matrix
count of words in vocabulary for each document

## pi
word topic distribution for each document (what probability does word belong to topic)

## gamma
topic distribution for each document


In [5]:
# set parameters
k = 2 # number of topics
v = 5 # number of words in vocabulary
d = 3 # number of documents in corpus
e_num_iter = 2 # number of iterations for e-step

In [6]:
alpha = np.array([1/k for i in range(k)])

In [7]:
alpha

array([0.5, 0.5])

In [8]:
beta = np.full((k, v), 1/k)

In [9]:
beta

array([[0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5]])

In [10]:
term_doc_matrix = np.random.randint(10, size=(d, v))

In [11]:
term_doc_matrix

array([[8, 6, 5, 5, 2],
       [7, 7, 7, 8, 6],
       [1, 7, 4, 5, 0]])

In [12]:
gamma = np.full((d, k), alpha)
for i in range(d):
    gamma[i] = gamma[i] + term_doc_matrix[i].sum()/k

In [13]:
gamma

array([[13.5, 13.5],
       [18. , 18. ],
       [ 9. ,  9. ]])

In [14]:
# check with sum of words in each doc
term_doc_matrix.sum(axis=1)

array([26, 35, 17])

In [19]:

for i in range(e_num_iter):
    p = []
    for j in range(d):
        p.append(((beta * term_doc_matrix[j]).T * np.exp(digamma(gamma[j])-digamma(gamma[j].sum()))).T)
    pi = np.array(p)
    for j in range(d):
        pi[j] = normalize(pi[j], 0)
    # update gamma
    for j in range(d):
        gamma[j] = alpha + pi[j].sum(axis=1)

In [20]:
gamma

array([[3. , 3. ],
       [3. , 3. ],
       [2.5, 2.5]])

# alpha maximization

In [33]:
gamma

array([[3. , 3. ],
       [3. , 3. ],
       [2.5, 2.5]])

In [23]:
from scipy.special import digamma, gamma as gamma_func
from scipy import optimize

In [38]:
def alpha_likelihood(alpha):
    l_a = None
    term_1 = d*np.log(gamma_func(alpha.sum()))
    term_2 = d*np.log(gamma_func(alpha)).sum()
    term_3 = 0
    # term_3
    for i in range(d): # of docs
        term_3 = term_3 + ((alpha - 1) * (digamma(gamma[i]) - digamma(gamma[i].sum()))).sum()
    l_a = (term_1 - term_2 + term_3) * -1 # to get maximum
    return l_a

In [43]:
optimize.minimize(alpha_likelihood, alpha)

      fun: -0.7288773730131606
 hess_inv: array([[4.95579503, 3.95579503],
       [3.95579503, 4.95579503]])
      jac: array([-8.34465027e-07, -8.34465027e-07])
  message: 'Optimization terminated successfully.'
     nfev: 27
      nit: 8
     njev: 9
   status: 0
  success: True
        x: array([2.81097128, 2.81097128])

In [44]:
alpha_likelihood(alpha)

1.064561963094976

In [45]:
alpha_likelihood(np.array([2.8, 2.8]))

-0.7288639206425014

In [50]:
alpha_likelihood(np.array([4, 4]))

-0.60716110110857